# Python语言与经济大数据分析

# 第5_2讲 网络爬虫进阶

In [ ]:
# 郭峰（上海财经大学投资系副教授）
#Email：guo.feng@mail.sufe.edu.cn
#2019-2-27
#本讲目录
#5.2.1 异步加载式网页的爬取:2345天气网
#5.2.2 表单交互（post方法）：环保部AQI

# 5.2.1 异步加载式网页的爬取

In [ ]:
#对于异步加载的网页，数据不在“网页源代码”
#例如简书网：https://www.jianshu.com/c/bDHhpK
#网页加载这些数据的过程也称为“逆行工程”
#以2345天气网来讲解这类网站的爬取方法，http://tianqi.2345.com/
#第一步，打开浏览器的开发者工具（部分电脑为F12键），选择Network选项
#第二步，刷新或者点击下一页，可以出现相关信息
#第三步，在headers部分可以看到请求的url，从中可以找到相关规律
#第四步，preview当中可以找到相关数据
#目前只搞定一个2345天气网的爬虫，罗攀著作提供的案例就没搞定。。。

In [ ]:
#### 【实际应用】2345天气网爬取
#先爬一个id列表
from urllib import request, parse
from urllib.request import urlopen
import time  
import re
import requests
import csv
head={}  #字典结构
head['User-Agent']='Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'
#读取id
path='D:/python/tianqi//' 
f=open(path+'cityid'+'.csv','a',encoding='gb2312') 
c=csv.writer(f)
#c.writerow(('city_id','city_1','city_2'))            
for id in range(72129,73000):       #最高到72129
    url='http://tianqi.2345.com/wea_history/'+str(id)+'.htm'
#        http://tianqi.2345.com/wea_history/71445.htm       #模仿此公式计算而来
    print(url)
    req=requests.get(url,headers=head)
    if req.status_code==200:
        resp=request.urlopen(url)
        html=resp.read()
        string=html.decode(encoding='gb2312',errors='ignore')  
#        print(string)
        city1=re.findall('<a href="/" title="天气预报">天气预报</a><span>&gt;</span><a title="(.*?)天气预报" href',string) 
#       <a href="/" title="天气预报">天气预报</a><span>&gt;</span><a title="青海天气预报" href="/qinghai_dz/30.htm" 
        city2=re.findall('id="lastBread">(.*?)历史天气</a>',string,re.S)    
#                         id="lastBread">德令哈历史天气</a>
        city=city1+city2           #这是个列表，每个元素是一串文字，而不是一个
        print(id,city1,city2)
        c.writerow((id,city1[0],city2[0]))
    else:
        continue
    time.sleep(2)           #爬取间隔数据
f.close()

In [ ]:
#【实际应用】2345天气网爬取
#正式爬取
from urllib import request, parse
from urllib.request import urlopen
import time  
import re
import requests
import csv
import http.cookiejar
path='D:/python/tianqi/' 
pathdata='D:/python/tianqi/data/'

head={}  #字典结构
head['User-Agent']='Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'

cityid=open(path+'cityidnew.csv','r')
cityid=cityid.read()
cityids=cityid.split('\n')
cityids=cityids[0:370]        #多出一行，末尾为空，搞不清楚原因，可能是上一行\n切割导致的 
    
cityname=open(path+'cityname.csv','r')
cityname=cityname.read()
citynames=cityname.split('\n')

month=open(path+'monthn.csv','r')
month=month.read()
mons=month.split('\n')
mons=mons[0:64]
print(len(cityids),len(mons))
#print(mons)
i=369
while i<len(cityids):   #不使用for语言，是为了更好地监控爬虫进程，以及出错后的后续跟进
    print(i,citynames[i])
    f=open(pathdata+'_'+str(i)+'_'+citynames[i]+'.csv','w+',encoding='gb2312') 
    c=csv.writer(f)
    c.writerow(('city_id','city_name','date','wenduh','wendul','tianqi','fengxiang','fengli','aqi'))
    for mon in mons:
        if mons.index(mon)<38:  #2016年3月前和后网址不太一样。
            url = "http://tianqi.2345.com/t/wea_history/js/"+cityids[i]+'_'+mon+'.js'
        else:
            url = "http://tianqi.2345.com/t/wea_history/js/"+mon+'/'+cityids[i]+'_'+mon+'.js'
#             http://tianqi.2345.com/t/wea_history/js/201803/71799_201803.js
        print(url)
        req=requests.get(url,headers=head)
        resp=request.urlopen(url)
        html=resp.read()
        string=html.decode(encoding='gb2312',errors='ignore')  
#            print(string)
        city=re.findall('city:\'(.*?)\',tqInfo',string)
        if mons.index(mon)<36:         #2015年12月前和后的数据不一样，之后有aqi，之前没有。
            data=re.findall('ymd:\'(.*?)\',bWendu:\'(.*?)\',yWendu:\'(.*?)\',tianqi:\'(.*?)\',fengxiang:\'(.*?)\',fengli:\'(.*?)\'}',string,re.S)
        else:
            data=re.findall('ymd:\'(.*?)\',bWendu:\'(.*?)\',yWendu:\'(.*?)\',tianqi:\'(.*?)\',fengxiang:\'(.*?)\',fengli:\'(.*?)\',aqi:\'(.*?)\',aqiInfo',string,re.S)
#        {city:'鼓楼',tqInfo:[{ymd:'2018-03-01',bWendu:'14℃',yWendu:'4℃',tianqi:'多云~小雨',fengxiang:'东北风',fengli:'4-5级',aqi:'91',aqiInfo:'良',aqiLevel:'2'}
        for d in range(0,len(data)):
            if mons.index(mon)<36:
                c.writerow((cityids[i],city[0],data[d][0],data[d][1],data[d][2],data[d][3],data[d][4],data[d][5]))
            else:
                c.writerow((cityids[i],city[0],data[d][0],data[d][1],data[d][2],data[d][3],data[d][4],data[d][5],data[d][6])) 
        time.sleep(0.5) #爬取间隔数据
    f.close()
    i=i+1

In [ ]:
#【实际应用】2018年5月和6月的2345天气网爬取
from urllib import request, parse
from urllib.request import urlopen
import time  
import re
import requests
import csv
import http.cookiejar
path='D:/python/tianqi/' 
pathdata='D:/python/tianqi/data2/'

head={}  #字典结构
head['User-Agent']='Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'

cityid=open(path+'cityidnew.csv','r')    #城市在天气网中的对应代码
cityid=cityid.read()
cityids=cityid.split('\n')
cityids=cityids[0:370]        #多出一行，末尾为空，搞不清楚原因，可能是上一行\n切割导致的 
    
cityname=open(path+'cityname.csv','r')   #城市名称，与cityid一一对应
cityname=cityname.read()
citynames=cityname.split('\n')

mons=['201805','201806']

f=open(pathdata+'mon1805'+'_'+'mon1806'+'.csv','w+',encoding='gb2312') 
c=csv.writer(f)
c.writerow(('city_id','city_name','date','wenduh','wendul','tianqi','fengxiang','fengli','aqi'))

i=0

while i<len(cityids):   #不使用for语言，是为了更好地监控爬虫进程，以及出错后的后续跟进
    print(i,citynames[i])
    for mon in mons:
        url = "http://tianqi.2345.com/t/wea_history/js/"+mon+'/'+cityids[i]+'_'+mon+'.js'
#             http://tianqi.2345.com/t/wea_history/js/201803/71799_201803.js
        print(url)
        req=requests.get(url,headers=head)
        resp=request.urlopen(url)
        html=resp.read()
        string=html.decode(encoding='gb2312',errors='ignore')  
#            print(string)
        city=re.findall('city:\'(.*?)\',tqInfo',string)
        data=re.findall('ymd:\'(.*?)\',bWendu:\'(.*?)\',yWendu:\'(.*?)\',tianqi:\'(.*?)\',fengxiang:\'(.*?)\',fengli:\'(.*?)\',aqi:\'(.*?)\',aqiInfo',string,re.S)
#        {city:'鼓楼',tqInfo:[{ymd:'2018-03-01',bWendu:'14℃',yWendu:'4℃',tianqi:'多云~小雨',fengxiang:'东北风',fengli:'4-5级',aqi:'91',aqiInfo:'良',aqiLevel:'2'}
        for d in range(0,len(data)):
            c.writerow((cityids[i],city[0],data[d][0],data[d][1],data[d][2],data[d][3],data[d][4],data[d][5],data[d][6])) 
        time.sleep(0.1) #爬取间隔数据
    i=i+1
f.close()
   

In [ ]:
#【实际应用】2345天气网个别地区的爬取,这些地区aqi数据一会有一会没有,
#或者早期样本缺失
#包括201805、201806两个年份
from urllib import request, parse
from urllib.request import urlopen
import time  
import re
import requests
import csv
import http.cookiejar
path='D:/python/tianqi/' 
pathdata='D:/python/tianqi/data2/'

head={}  #字典结构
head['User-Agent']='Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/55.0.2883.87 Safari/537.36'


cityid='71444'
cityname='三沙'

month=open(path+'monthn2.csv','r')
month=month.read()
mons=month.split('\n')
mons=mons[0:66]
#print(mons)

f=open(pathdata+'_'+cityname+'.csv','w+',encoding='gb2312',newline='') 
c=csv.writer(f)
c.writerow(('city_id','city_name','date','wenduh','wendul','tianqi','fengxiang','fengli','aqi'))
for mon in mons:
    if mons.index(mon)<38:  #2016年3月前和后网址不太一样。
        url = "http://tianqi.2345.com/t/wea_history/js/"+cityid+'_'+mon+'.js'
    else:
        url = "http://tianqi.2345.com/t/wea_history/js/"+mon+'/'+cityid+'_'+mon+'.js'
#       http://tianqi.2345.com/t/wea_history/js/201803/71799_201803.js
    print(url)
    try:
        req=requests.get(url,headers=head)
        resp=request.urlopen(url)
        html=resp.read()
        string=html.decode(encoding='gb2312',errors='ignore')  
#       print(string)
        city=re.findall('city:\'(.*?)\',tqInfo',string)
        data=re.findall('ymd:\'(.*?)\',bWendu:\'(.*?)\',yWendu:\'(.*?)\',tianqi:\'(.*?)\',fengxiang:\'(.*?)\',fengli:\'(.*?)\'',string,re.S)
#        {city:'鼓楼',tqInfo:[{ymd:'2018-03-01',bWendu:'14℃',yWendu:'4℃',tianqi:'多云~小雨',fengxiang:'东北风',fengli:'4-5级',aqi:'91',aqiInfo:'良',aqiLevel:'2'}
    #    city:'资阳',tqInfo:[{ymd:'2017-05-01',bWendu:'30℃',yWendu:'21℃',tianqi:'多云~雷阵雨',fengxiang:'无持续风向',fengli:'微风'}
        for d in range(0,len(data)):
            c.writerow((cityid,city[0],data[d][0],data[d][1],data[d][2],data[d][3],data[d][4],data[d][5]))
    except:
        continue
    time.sleep(0.1) #爬取间隔数据
f.close()


In [1]:
#天气数据合并
import os
import csv
path='D:/python/tianqi/' 
pathdata='D:/python/tianqi/datatotal/'
fn=open(path+'tianqi.csv','w+',encoding='gb2312',newline='') 
cw=csv.writer(fn)
cw.writerow(('city_id','city_name', 'date','wenduh','wendul','tianqi','fengxiang','fengli','aqi'))

files= os.listdir(pathdata)  #得到文件夹下的所有文件名称  
for file in files:    #遍历文件夹 
    f=open(pathdata+file,'r',encoding='gb2312') 
    cr=csv.reader(f)
    for row in cr:    
        if row==[] or row[0]=='city_id':
            continue
        else:
            if len(row)==8:
                rown=[row[0],row[1],row[2],row[3][:-1],row[4][:-1],row[5],row[6],row[7]]
            else:
                rown=[row[0],row[1],row[2],row[3][:-1],row[4][:-1],row[5],row[6],row[7],row[8]]
            cw.writerow(rown)
    f.close()
fn.close()   


In [ ]:
#中间步骤
import csv
pathdata='D:/python/tianqi/data/'
f=open(pathdata+'_0_北京.csv','r',encoding='gb2312') 
cr = csv.reader(f)
for row in cr:
    if row==[] or row[0]=='city_id':
        continue
    else:
        print(row[0])
     

# 5.2.2 表单交互：环保部AQI

In [ ]:
# 【实际应用】爬取环保部数据中心AQI数据，post格式。（！！！只能缓慢爬取，容易被封）
import urllib
import http.cookiejar
import re
import csv
import time
import pandas as pd
import random
pathdata='D:/python/aqi/data/'

#整理header
header = {    
"Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Language":"zh,zh-CN;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection":"keep-alive",
    "Host":"datacenter.mep.gov.cn",
    "Origin": "http://datacenter.mep.gov.cn",        
    "Referer":"http://datacenter.mep.gov.cn/websjzx/report!list.vm?xmlname=1512478367400&roleType=CFCD2084&permission=null",
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.84 Safari/537.36",
    "Upgrade-Insecure-Requests":1 ,
    }
dd=open('D:/python/aqi/date.csv','r')
dates=dd.read().split('\n')
d=15
break_flag=False
while d<len(dates):
    f=open(pathdata+'_'+dates[d]+'_aqi.csv','w+',encoding='gb2312') 
    c=csv.writer(f)
    c.writerow(('city_name','aqi','level','prime','date'))
    print(d,dates[d])
    for j in range(37):
        url = "http://114.251.10.135/websjzx/report/list.vm"
        postdata =urllib.parse.urlencode({
            "pageNum": j,
            "xmlname":"1512478367400",
            "queryflag":"close",
            "customquery": "false",  
            "isdesignpatterns":"false",
            "roleType": "CFCD2084",
            "permission":"0",   
            "V_DATE":dates[d],
            "inPageNo":"1"
        }).encode('utf-8')
        req = urllib.request.Request(url,postdata,header)
        cj = http.cookiejar.CookieJar()
        opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
        r = opener.open(req)
        str1=r.read().decode('utf-8') 
        cityname=re.findall("style='color:blue;'>(.*?)</a></td>",str1)
        aqi=re.findall('<span style=\'[^\']+\'>([0-9]+)',str1)             #这一个始终没看懂，来自于周耿讲义
        prime=re.findall('colid="5"  rowspan="1"  colspan="1" style="text-align:center;WORD-WRAP: break-word;">(.*?)</td>',str1)
        level=re.findall("<span style=\'[^\']+\'>([\D]+)</span></td>",str1)
        date=re.findall('colid="6"  rowspan="1"  colspan="1" style="text-align:center;WORD-WRAP: break-word;">(.*?)</td>',str1)
        for i in range(len(aqi)):
            c.writerow((cityname[i],aqi[i],level[i],prime[i],date[i]))
        time.sleep(random.choice([15,10,5,20]))
        if aqi==[]:
            break_flag=True
            print('break_flag is',break_flag)
            break
        else:
            print(date[i],j,'keep going')
            continue
    if break_flag==True:
        break
    else:
        d=d+1
    f.close()
    time.sleep(60)